In [19]:
import os
import re
from pathlib import Path
from reportlab.pdfgen import canvas
from reportlab.pdfbase import pdfmetrics
from reportlab.pdfbase.ttfonts import TTFont
from reportlab.lib.units import inch
from reportlab.lib.pagesizes import landscape, A4
import textwrap

# 设置页面参数
PAGE_WIDTH, PAGE_HEIGHT = landscape(A4)
FONT_SIZE_WORD = 28
FONT_SIZE_CHINESE = 22
LINE_SPACING = 40
CHINESE_LINE_LIMIT = 15  # 每行最多15个中文字符

In [ ]:
"C:\Users\TJ002\Desktop\Work\Learning-CLUES\Language\word list\dejavu-fonts-ttf-2.37\ttf\DejaVuSans.ttf"

In [23]:
# 字体路径（修改为你解压的字体位置）
DEJAVU_FONT_PATH = r"C:\Users\TJ002\Desktop\Work\Learning-CLUES\Language\word list\dejavu-fonts-ttf-2.37\ttf\DejaVuSans.ttf"
DEJAVU_FONT_NAME = "DejaVu"

# 中文优先字体列表（按系统平台推荐）
CHINESE_FONT_CANDIDATES = [
    "Microsoft YaHei", "SimSun",       # Windows
    "PingFang SC", "STHeiti",          # macOS
    "Noto Sans CJK SC", "WenQuanYi Zen Hei"  # Linux
]

# 注册 DejaVu 用于英文 + 音标
pdfmetrics.registerFont(TTFont(DEJAVU_FONT_NAME, DEJAVU_FONT_PATH))
print(f"✅ 英文字体: {DEJAVU_FONT_NAME}")

# 查找并注册中文字体
def find_chinese_font(font_list):
    for name in font_list:
        try:
            path = fm.findfont(name, fallback_to_default=False)
            if os.path.exists(path):
                pdfmetrics.registerFont(TTFont(name, path))
                print(f"✅ 中文字体: {name}")
                return name
        except:
            continue
    raise RuntimeError("❌ 未找到可用中文字体")

CHINESE_FONT_NAME = find_chinese_font(CHINESE_FONT_CANDIDATES)

✅ 英文字体: DejaVu
✅ 中文字体: Microsoft YaHei


In [24]:
def parse_line(line):
    """
    解析格式：英文 + /音标/ + 中文释义
    """
    match = re.match(r'^(.+?)\s*/([^/]+)/\s*(.*)$', line)
    if not match:
        return None
    word_part, phonetic, chinese = match.groups()
    word_clean = word_part.replace("*", "").strip()
    eng_line = f"{word_clean}   /{phonetic}/"

    chinese_lines = []
    for chunk in chinese.strip().split("；"):
        chunk = chunk.strip()
        if chunk:
            wrapped = textwrap.wrap(chunk, width=CHINESE_LINE_LIMIT)
            chinese_lines.extend(wrapped)

    return eng_line, chinese_lines

def draw_centered_text(c, lines, start_y, font_name, font_size):
    for i, line in enumerate(lines):
        text_width = c.stringWidth(line, font_name, font_size)
        x = (PAGE_WIDTH - text_width) / 2
        y = start_y - i * LINE_SPACING
        c.setFont(font_name, font_size)
        c.drawString(x, y, line)

def txt_to_pdf(txt_path, output_dir):
    filename = Path(txt_path).stem
    pdf_path = os.path.join(output_dir, f"{filename}.pdf")

    with open(txt_path, 'r', encoding='utf-8') as f:
        raw_lines = [line.strip() for line in f if line.strip()]

    c = canvas.Canvas(pdf_path, pagesize=(PAGE_WIDTH, PAGE_HEIGHT))

    for raw in raw_lines:
        result = parse_line(raw)
        if not result:
            continue
        eng_line, chinese_lines = result

        total_height = (1 + len(chinese_lines)) * LINE_SPACING
        start_y = (PAGE_HEIGHT + total_height) / 2 - LINE_SPACING

        draw_centered_text(c, [eng_line], start_y, DEJAVU_FONT_NAME, FONT_SIZE_WORD)
        draw_centered_text(c, chinese_lines, start_y - LINE_SPACING, CHINESE_FONT_NAME, FONT_SIZE_CHINESE)

        c.showPage()

    c.save()
    print(f"📄 已生成: {pdf_path}")

def process_all_wordlists(folder):
    output_dir = os.path.join(folder, "PDF_output_final_fonts")
    os.makedirs(output_dir, exist_ok=True)

    for file in os.listdir(folder):
        if file.startswith("word list") and file.endswith(".txt"):
            txt_path = os.path.join(folder, file)
            txt_to_pdf(txt_path, output_dir)

In [28]:
# 指定包含 "word list*.txt" 的文件夹路径
folder = r"C:\Users\TJ002\Desktop\Work\Learning-CLUES\Language\word list"

# 调用处理函数
process_all_wordlists(folder)


📄 已生成: C:\Users\TJ002\Desktop\Work\Learning-CLUES\Language\word list\PDF_output_final_fonts\word list24.pdf
📄 已生成: C:\Users\TJ002\Desktop\Work\Learning-CLUES\Language\word list\PDF_output_final_fonts\word list25.pdf
📄 已生成: C:\Users\TJ002\Desktop\Work\Learning-CLUES\Language\word list\PDF_output_final_fonts\word list26.pdf
📄 已生成: C:\Users\TJ002\Desktop\Work\Learning-CLUES\Language\word list\PDF_output_final_fonts\word list27.pdf
📄 已生成: C:\Users\TJ002\Desktop\Work\Learning-CLUES\Language\word list\PDF_output_final_fonts\word list28.pdf
📄 已生成: C:\Users\TJ002\Desktop\Work\Learning-CLUES\Language\word list\PDF_output_final_fonts\word list29.pdf
📄 已生成: C:\Users\TJ002\Desktop\Work\Learning-CLUES\Language\word list\PDF_output_final_fonts\word list30.pdf
📄 已生成: C:\Users\TJ002\Desktop\Work\Learning-CLUES\Language\word list\PDF_output_final_fonts\word list31.pdf
📄 已生成: C:\Users\TJ002\Desktop\Work\Learning-CLUES\Language\word list\PDF_output_final_fonts\word list32.pdf
📄 已生成: C:\Users\TJ002\Deskto